# Extração de dados do Github

Pesquisaremos por iniciativas/projetos que utilizam Dados Abertos Governamentais através da [API do Github](https://developer.github.com/v3/)

A partir das apontamentos feitos na qualificação desse projeto de pesquisa, entendemos que caracterizar toda a comunidade é um escopo muito abrangente e de difícil validação.
A ideia é reduzir esse escopo para assim poder avaliar melhor os seus resultados, por exemplo verificar se os projetos referências nessas áreas aparecem nos repositórios extraídos no Github.

Vamos testar a extração de alguns assuntos e palavras chave:

Educação
- Enem
- Prova Brasil
- Enade
- MEC
- Ies
- Inep

Saúde:
- Sus
- Saúde
- Violência
- Saúde da mulher

In [2]:
import requests
import pandas as pd
import time
import logging

In [4]:
strings_edu = [
    'dados enem',
    'dados enade',
    'dados prova brasil',
    'dados ies',
    'dados educacao',
    'dados mec',
    'dados inep',
    'dados escola',
    'dados escola inep',
    'microdados inep'
]

In [5]:
strings_saude = [
    'dados sus',
    'dados saúde',
    'dados saude da mulher',
]

Configuração para gerar arquivo de log

In [6]:
logging.basicConfig(level=logging.DEBUG, 
                    filename="log_file.txt", 
                    filemode="a+",
                    format="%(asctime)s - %(levelname)s - %(funcName)s - %(message)s")

logging.info("Extração de dados do Github")

In [7]:
search_strings = [
            'dados abertos',
            'dados abertos brasil',
            'dados abertos governo',
            'dados abertos governamentais',
            'dados governamentais',
            'dados publicos abertos',
            'dados do governo',
            'analise de dados do governo',
            'analise de dados governamentais',
            'portal de dados do governo',
            'portal de dados governamentais',
            'portal publico do governo',
            'portal de dados abertos do governo',
        ]

Para a acesso a alguns recursos da API do github é preciso se autenticar, como aumentar o limite de requisições. Informações sobre autenticação podem ser encontradas [aqui](https://developer.github.com/v3/#authentication).

In [8]:
credentials = ('<user_name>','<token>')

Limite de requisições sem autenticação

In [9]:
t = requests.get('https://api.github.com/rate_limit')
t.json()

{'resources': {'core': {'limit': 60, 'remaining': 60, 'reset': 1584116335},
  'search': {'limit': 10, 'remaining': 10, 'reset': 1584112795},
  'graphql': {'limit': 0, 'remaining': 0, 'reset': 1584116335},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1584116335},
  'source_import': {'limit': 5, 'remaining': 5, 'reset': 1584112795}},
 'rate': {'limit': 60, 'remaining': 60, 'reset': 1584116335}}

Limite de requisições com autenticação

In [10]:
t = requests.get('https://api.github.com/rate_limit', auth=credentials)
t.json()

{'resources': {'core': {'limit': 5000, 'remaining': 5000, 'reset': 1584116338},
  'search': {'limit': 30, 'remaining': 30, 'reset': 1584112798},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1584116338},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1584116338},
  'source_import': {'limit': 100, 'remaining': 100, 'reset': 1584112798}},
 'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1584116338}}

Verificando limitação de extração de dados da API

In [11]:
page_35 = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(page_35, auth=credentials)
t.json()

{'message': 'Only the first 1000 search results are available',
 'documentation_url': 'https://developer.github.com/v3/search/'}

Informações sobre a ferramenta de pesquisa da API podem ser encontradas [aqui](https://developer.github.com/v3/search/)

In [12]:
url_base = 'https://api.github.com/search/repositories?q='

Podemos adicionar uma ordenação nos resultados, como quantidade de _stars_ de forma descrescente.

In [13]:
sort = '&sort=stars&order=desc'

## Extraindo informações gerais

In [14]:
def extract_results(data):
    
    items_list = []
    
    for item in data.get('items', None):
        
        
        item_dict = {
                'id': item.get('id'),
                'full_name': item.get('full_name', None),
                'description': item.get('description', None),      
                'owner_type': item.get('owner').get('type', None),
                'owner_api_url': item.get('owner').get('url', None),
                'owner_url': item.get('owner').get('html_url', None),
                'api_url': item.get('url', None),
                'url': item.get('html_url', None),
                'fork': item.get('fork', None),
                'created_at': item.get('created_at', None),
                'updated_at': item.get('updated_at', None),
                'pushed_at': item.get('pushed_at', None),
                'size': item.get('size', None),
                'stargazers_count': item.get('stargazers_count', None),
                'language': item.get('language', None),
                'has_issues': item.get('has_issues', None),
                'has_wiki': item.get('has_wiki', None),
                'forks_count': item.get('forks_count', None),
                'forks': item.get('forks', None),
                'open_issues': item.get('open_issues', None),
                'license': item.get('license').get('name', None) if item.get('license', None) else None,
                'timestamp_extract': str(time.time()).split('.')[0]
        }

        items_list.append(item_dict)
            
    return items_list

In [15]:
def scroll_pages(url):
        
    results = requests.get(url, auth=credentials)    
    data = results.json()
    total = data.get('total_count', None)
        
    logging.info("Foram encontrados {0} resultados. Extraindo...".format(total))
        
    items_list = []
    items_list = extract_results(data)
        
    iterations = total // 30 
    
    for iteracao in range(0, iterations):        
        header = results.links
        
        if header.get('next', False):
            next_url = header.get('next').get('url')
                        
            results = requests.get(next_url, auth=credentials)
            data = results.json()
            
            items_list = items_list + extract_results(data)
    
    return items_list

In [16]:
%%time

items_list = []
repositories_df = None

for string in strings_edu:
    url = url_base + string + sort
    
    logging.info("Pesquisando repositórios para a string: '{0}'".format(string))
    
    t = scroll_pages(url)
    items_list = items_list + t
    print(string)
    print(len(t))
        
repositories_df = pd.DataFrame(items_list)

dados enem
49
dados enade
17
dados prova brasil
2
dados ies
19
dados educacao
54
dados mec
10
dados inep
43
dados escola
137
dados escola inep
9
microdados inep
20
CPU times: user 548 ms, sys: 32.1 ms, total: 580 ms
Wall time: 24.4 s


In [18]:
repositories_df.head(3)

,id,full_name,description,owner_type,owner_api_url,owner_url,api_url,url,fork,created_at,...,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,license,timestamp_extract
0,224536388,jesspiires/microdadosEnem_Deficit_Atencao,Análise dos Micro Dados dos alunos que fizeram...,User,https://api.github.com/users/jesspiires,https://github.com/jesspiires,https://api.github.com/repos/jesspiires/microd...,https://github.com/jesspiires/microdadosEnem_D...,False,2019-11-27T23:53:57Z,...,107,3,Jupyter Notebook,True,True,1,1,0,None,1584112727
1,52244477,sombriks/microdados-brutos,script para carga de dados brutos do enem de 1...,User,https://api.github.com/users/sombriks,https://github.com/sombriks,https://api.github.com/repos/sombriks/microdad...,https://github.com/sombriks/microdados-brutos,False,2016-02-22T03:30:11Z,...,13411,3,JavaScript,True,True,0,0,0,None,1584112727
2,155470617,leonardojs1981/analise_enem_2,Código para análise de dados do ENEM e Censo E...,User,https://api.github.com/users/leonardojs1981,https://github.com/leonardojs1981,https://api.github.com/repos/leonardojs1981/an...,https://github.com/leonardojs1981/analise_enem_2,False,2018-10-30T23:38:24Z,...,123,2,Jupyter Notebook,True,True,4,4,0,None,1584112727


Quantidade de resultados:

In [19]:
len(repositories_df)

360

Retirando registros duplicados visto que palavras de busca diferentes podem levar a um mesmo repositório.

In [22]:
repositories_df = repositories_df.drop_duplicates(['id', 'api_url'])

In [23]:
len(repositories_df)

311

In [24]:
repositories_df.describe()

,id,size,stargazers_count,forks_count,forks,open_issues
count,3.110000e+02,311.000000,311.000000,311.000000,311.000000,311.000000
mean,1.584593e+08,10354.832797,0.565916,0.405145,0.405145,0.318328
std,6.710318e+07,35552.879356,2.940484,1.413018,1.413018,2.390496
min,2.579957e+06,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.097918e+08,32.000000,0.000000,0.000000,0.000000,0.000000
50%,1.758186e+08,296.000000,0.000000,0.000000,0.000000,0.000000
75%,2.186834e+08,3474.500000,0.000000,0.000000,0.000000,0.000000
max,2.465781e+08,306083.000000,47.000000,17.000000,17.000000,35.000000


Quantidade de colunas:

In [25]:
len(repositories_df.columns)

22

In [31]:
repositories_df = repositories_df.sort_values('stargazers_count', ascending=False)

In [35]:
repositories_df.head(1)

,id,full_name,description,owner_type,owner_api_url,owner_url,api_url,url,fork,created_at,...,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,license,timestamp_extract
87,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://api.github.com/repos/prefeiturasp/dado...,https://github.com/prefeiturasp/dados-educacao,False,2016-11-10T13:35:40Z,...,2737,47,Jupyter Notebook,True,True,17,17,1,None,1584112734


## Extraindo _Commits_, _Contributors_ e dados do _Owner_

In [36]:
repo_copia = repositories_df

In [37]:
def extract_commits(url_repo):
    
    commits_url = url_repo + '/commits'  
    results = requests.get(commits_url, auth=credentials)
    
    if results.status_code == 409:
        return None
    
    commits = len(results.json())

    header = results.links
    
    while header.get('next', False):
        next_url = header.get('next').get('url')        
        results = requests.get(next_url, auth=credentials)
        commits = commits + len(results.json())    
        header = results.links


    return commits

In [38]:
def extract_contributors(url_repo):
    
    contributors_url = url_repo + '/contributors'
    results = requests.get(contributors_url, auth=credentials)
    
    if results.status_code == 204:
        return None
    
    contributors = len(results.json())

    header = results.links
    
    while header.get('next', False):
        next_url = header.get('next').get('url')
        results = requests.get(next_url, auth=credentials)
        contributors = contributors + len(results.json())
        header = results.links
    
    return contributors

In [39]:
def extract_owner_data(owner_api_url):
    
    results = requests.get(owner_api_url, auth=credentials)
    data = results.json()

    owner_data = {
        'owner_location': data.get('location', None),
        'owner_email': data.get('email', None),
        'owner_blog': data.get('blog', None),
        'owner_name': data.get('name', None)
    }
    
    return owner_data

In [40]:
%%time
urls = repositories_df['api_url']

for url in urls:

    owner_api_url = repositories_df.loc[repositories_df["api_url"] == url]['owner_api_url'].item()
    owner_data = extract_owner_data(owner_api_url)
    commits = extract_commits(url)
    contributors = extract_contributors(url)
    
    logging.info("Repositório: {0}".format(url))
    logging.info("Tem {0} Commits - {1} Contributors".format(commits,contributors))
    logging.info("Owner location: {0}".format(owner_data.get('owner_location')))

    repositories_df.loc[repositories_df["api_url"] == url, 'commits'] = commits
    repositories_df.loc[repositories_df["api_url"] == url, 'contributors'] = contributors
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_location'] = owner_data.get('owner_location')
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_email'] = owner_data.get('owner_email')
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_blog'] = owner_data.get('owner_blog')
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_name'] = owner_data.get('owner_name')

CPU times: user 33.5 s, sys: 1.2 s, total: 34.7 s
Wall time: 8min 51s


Agora devemos ter mais 6 colunas

In [41]:
len(repositories_df.columns)

28

In [43]:
repositories_df.head(3)

,id,full_name,description,owner_type,owner_api_url,owner_url,api_url,url,fork,created_at,...,forks,open_issues,license,timestamp_extract,commits,contributors,owner_location,owner_email,owner_blog,owner_name
87,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://api.github.com/repos/prefeiturasp/dado...,https://github.com/prefeiturasp/dados-educacao,False,2016-11-10T13:35:40Z,...,17,1,None,1584112734,18.0,2.0,"São Paulo, SP",tecnologia@prefeitura.sp.gov.br,http://www.capital.sp.gov.br,Prefeitura Municipal de São Paulo
151,19895876,inepdadosabertos/api,API de dados abertos para o INEP,Organization,https://api.github.com/users/inepdadosabertos,https://github.com/inepdadosabertos,https://api.github.com/repos/inepdadosabertos/api,https://github.com/inepdadosabertos/api,False,2014-05-17T20:45:05Z,...,4,2,GNU General Public License v2.0,1584112738,29.0,2.0,None,None,,None
89,215114032,Explicador/MozAPI,"API com diversos dados de Moz, incluindo dados...",Organization,https://api.github.com/users/Explicador,https://github.com/Explicador,https://api.github.com/repos/Explicador/MozAPI,https://github.com/Explicador/MozAPI,False,2019-10-14T18:12:48Z,...,0,2,MIT License,1584112734,5.0,1.0,"Maputo, Mozambique",info@explicador.co.mz,https://explicador.co.mz,Explicador


Conferindo valores nulos

In [44]:
len(repositories_df.loc[repositories_df['commits'].isnull()][['api_url', 'commits', 'contributors']])

16

Alguns repositórios realmente não tem nenhum commit como o [ccdpoa/ocida](https://github.com/ccdpoa/ocida).

In [55]:
repositories_df.loc[repositories_df['contributors'].isnull()][['id', 'url', 'api_url', 'commits', 'contributors']]

,id,url,api_url,commits,contributors
252,231368239,https://github.com/mario-rutman/ENTREVISTANDO-...,https://api.github.com/repos/mario-rutman/ENTR...,NaN,NaN
258,138407007,https://github.com/luisapanico/Gestores-Escolares,https://api.github.com/repos/luisapanico/Gesto...,NaN,NaN
272,115632691,https://github.com/RodrigoBarcelos/AgendaEscol...,https://api.github.com/repos/RodrigoBarcelos/A...,NaN,NaN
219,53981376,https://github.com/antonionei/dadosEscolas,https://api.github.com/repos/antonionei/dadosE...,NaN,NaN
223,212176337,https://github.com/SavioArthur/Projeto_Escola,https://api.github.com/repos/SavioArthur/Proje...,NaN,NaN
225,207171598,https://github.com/EdulinkBR/Escola,https://api.github.com/repos/EdulinkBR/Escola,NaN,NaN
231,210155655,https://github.com/juservi/escolas_brasileiras,https://api.github.com/repos/juservi/escolas_b...,NaN,NaN
347,123939819,https://github.com/pessoagabi/inep,https://api.github.com/repos/pessoagabi/inep,NaN,NaN
355,155280861,https://github.com/jhcf/exploradorinep,https://api.github.com/repos/jhcf/exploradorinep,NaN,NaN
281,131538999,https://github.com/gustavolaine/DS4A,https://api.github.com/repos/gustavolaine/DS4A,NaN,NaN


In [56]:
repositories_df = repositories_df.loc[repositories_df['contributors'].notnull()]
len(repositories_df)

295

Salvando os repositórios

In [57]:
repositories_df.to_csv('../data/repositories_edu.csv', index=False)

## Extraindo contribuidores dos repositórios

In [58]:
def get_contributors(data, repo_data):

    list_contributors = []

    for item in data:        
        contributor = {
            'repo_id': repo_data.get('repo_id', None),
            'repo_name': repo_data.get('repo_name', None),
            'repo_url': repo_data.get('repo_url', None),
            'repo_api_url': repo_data.get('repo_api_url', None),
            'contributor_id': item.get('id', None),
            'contributor_login': item.get('login', None),
            'contributor_type': item.get('type', None),
            'contributor_url': item.get('html_url', None),
            'contributor_api_url': item.get('url', None),
            'timestamp_extract': str(time.time()).split('.')[0]
        }

        list_contributors.append(contributor)

    return list_contributors

In [59]:
def scroll_contributors(url, repo_data):

    list_contributors = []
    results = requests.get(url, auth=credentials)
    
    if results.status_code is 204:
        return None
    
    data = results.json()
    list_contributors = get_contributors(data, repo_data)
    header = results.links
    
    while header.get('next', False):
        
        next_url = header.get('next').get('url')            
        results = requests.get(next_url, auth=credentials)
        data = results.json()
        list_contributors = list_contributors + get_contributors(data, repo_data)  
        header = results.links
        
    return list_contributors

In [60]:
def search_contributors(repositories_df):
    
    urls = repositories_df['api_url']
    list_contributors_all_repo = []
    
    for url in urls:
        logging.info('Extraindo contribuidores de: {0}'.format(url))
        
        repo_data = {
                'repo_id': repositories_df.loc[repositories_df["api_url"] == url, 'id'].values[0],
                'repo_name': repositories_df.loc[repositories_df["api_url"] == url, 'full_name'].values[0],
                'repo_url': repositories_df.loc[repositories_df["api_url"] == url, 'url'].values[0],
                'repo_api_url': url,
            }
        
        url_contributors = url + '/contributors'        
        contributors = scroll_contributors(url_contributors, repo_data)
        
        if contributors:
            list_contributors_all_repo = list_contributors_all_repo + contributors
    
    contributors_df = pd.DataFrame(list_contributors_all_repo)     
        
    return contributors_df

In [61]:
%%time
contributors_df = search_contributors(repositories_df)

CPU times: user 8.83 s, sys: 335 ms, total: 9.16 s
Wall time: 2min 3s


In [62]:
contributors_df.head(3)

,repo_id,repo_name,repo_url,repo_api_url,contributor_id,contributor_login,contributor_type,contributor_url,contributor_api_url,timestamp_extract
0,73385196,prefeiturasp/dados-educacao,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,2335525,campagnucci,User,https://github.com/campagnucci,https://api.github.com/users/campagnucci,1584116614
1,73385196,prefeiturasp/dados-educacao,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,28656406,tutss,User,https://github.com/tutss,https://api.github.com/users/tutss,1584116614
2,19895876,inepdadosabertos/api,https://github.com/inepdadosabertos/api,https://api.github.com/repos/inepdadosabertos/api,7613695,NandaBeirao,User,https://github.com/NandaBeirao,https://api.github.com/users/NandaBeirao,1584116615


Verificando se há contribuidores repetidos para um mesmo repositório.

In [63]:
contributors_df[contributors_df.duplicated(['contributor_id', 'repo_id'])]

,repo_id,repo_name,repo_url,repo_api_url,contributor_id,contributor_login,contributor_type,contributor_url,contributor_api_url,timestamp_extract


Salvando dataframe com mapeamento de repositórios e contribuidores.

In [64]:
contributors_df.to_csv('../data/contributors_edu.csv', index=False)